In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

ImportError: No module named 'folium'

In [2]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

Data downloaded!


In [3]:

with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)['features']

In [4]:

# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

In [5]:
for data in newyork_data:
    print(data['properties']['borough'])
    print(data['properties']['name'])
    borough =data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    print(neighborhood_lon)
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)
    
neighborhoods.head()

Bronx
Wakefield
-73.84720052054902
Bronx
Co-op City
-73.82993910812398
Bronx
Eastchester
-73.82780644716412
Bronx
Fieldston
-73.90564259591682
Bronx
Riverdale
-73.9125854610857
Bronx
Kingsbridge
-73.90281798724604
Manhattan
Marble Hill
-73.91065965862981
Bronx
Woodlawn
-73.86731496814176
Bronx
Norwood
-73.8793907395681
Bronx
Williamsbridge
-73.85744642974207
Bronx
Baychester
-73.83579759808117
Bronx
Pelham Parkway
-73.85475564017999
Bronx
City Island
-73.78648845267413
Bronx
Bedford Park
-73.8855121841913
Bronx
University Heights
-73.9104159619131
Bronx
Morris Heights
-73.91967159119565
Bronx
Fordham
-73.89642655981623
Bronx
East Tremont
-73.88735617532338
Bronx
West Farms
-73.87774474910545
Bronx
High  Bridge
-73.9261020935813
Bronx
Melrose
-73.90942160757436
Bronx
Mott Haven
-73.91609987487575
Bronx
Port Morris
-73.91322139386135
Bronx
Longwood
-73.8957882009446
Bronx
Hunts Point
-73.88331505955291
Bronx
Morrisania
-73.90150648943059
Bronx
Soundview
-73.86574609554924
Bronx
Clason Po

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [6]:
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


In [7]:

response = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text

from bs4 import BeautifulSoup
soup =BeautifulSoup(response,'lxml')
table =soup.tbody
rows = table.findAll('tr')
columns = ['PostCode', 'Borough', 'Neighborhood']
list=[]
df={}
for row in rows[1:]:
    cells = row.findAll('td')
    if (cells[1].text == 'Not assigned'):
     continue  
    postCode= cells[0].text
    borough = cells[1].text
    neigh = cells[2].text.strip()
    if (neigh == 'Not assigned'):
        neigh = borough
    df={
        'PostCode': postCode,
        'Borough' : borough,
        'Neighborhood'   : neigh,
     }
    list.append(df)
    
Neigh= pd.DataFrame(list, columns=columns)
Neigh = Neigh.groupby(['PostCode','Borough'])['Neighborhood'].apply(','.join).reset_index()


es = pd.read_csv('Geospatial_Coordinates.csv')

Neigh = pd.concat([Neigh,es['Latitude'],es['Longitude']],axis=1)
array=['Downtown Toronto','East Toronto', 'Central Toronto', 'West Toronto']
toronto_data = Neigh[Neigh['Borough'].isin(array)].reset_index(drop=True)
toronto_data.head()
    
toronto_data.drop(columns=['PostCode'],inplace=True)
toronto_data.reset_index(drop=True).head()

manhattan_data = manhattan_data.append(toronto_data)

manhattan_data = pd.concat([manhattan_data,toronto_data]).drop_duplicates().reset_index(drop=True)

manhattan_data

FileNotFoundError: File b'Geospatial_Coordinates.csv' does not exist

In [8]:
address = 'Manhattan, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Manhattan are 40.7900869, -73.9598295.


In [9]:
manhattan_data.loc[0, 'Neighborhood']

'Marble Hill'

In [10]:
neighborhood_latitude = manhattan_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = manhattan_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = manhattan_data.loc[0, 'Neighborhood'] # neighborhood name

print(neighborhood_name)
print(neighborhood_latitude)

Marble Hill
40.8765507788


In [11]:

# type your answer here
manhattan_data.head()
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

NameError: name 'CLIENT_ID' is not defined

In [12]:
results = requests.get(url).json()
results

NameError: name 'url' is not defined

In [13]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [14]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

NameError: name 'results' is not defined